In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
import os
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from skimage import io
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
class veri(Dataset): 
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations=pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.annotations)
    def __getitem__(self, index):
       img_path=os.path.join(self.root_dir,self.annotations.iloc[index,0])
       image=io.imread(img_path)
       y_label=torch.tensor(int(self.annotations.iloc[index,1]))
       
       if self.transform:
              image=self.transform(image)
              return (image,y_label)

In [3]:
dataset=veri(csv_file="D:\\udemy_data\\f1_classification\\f111.csv",root_dir="D:\\udemy_data\\f1_classification",
transform=torchvision.transforms.Compose([ 
    transforms.ToTensor(),
    transforms.Resize(size=(28,28)),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
]))

In [4]:
train_set,test_set=torch.utils.data.random_split(dataset,[200,79])
train_loader=DataLoader(dataset=train_set,batch_size=1,shuffle=False)
test_loader=DataLoader(dataset=test_set,batch_size=1,shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        #conv katmanları:
        self.conv1=nn.Conv2d(in_channels=3,out_channels=4,kernel_size=(5,5)) 
        self.conv2=nn.Conv2d(in_channels=4,out_channels=8,kernel_size=(3,3))
        self.conv3=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(2,2))
        self.conv4=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=(2,2))
        #max pooling katmanı:

        self.max=nn.MaxPool2d(kernel_size=(2,2)) 

       
        self.func=nn.ELU()

       
        self.fc1=nn.Linear(in_features=32,out_features=50)
        self.fc2=nn.Linear(in_features=50,out_features=50)
        self.fc3=nn.Linear(in_features=50,out_features=100)
        self.fc4=nn.Linear(in_features=100,out_features=4)

    def forward(self,x):
         x=self.conv1(x) 
         x=self.func(x)
         x=self.max(x)

         x=self.conv2(x)
         x=self.func(x)
         x=self.max(x)

         x=self.conv3(x)
         x=self.func(x)
         x=self.max(x)

         x=self.conv4(x)
         x=self.func(x)

         x=x.view(x.size(0),-1) 

         x=self.fc1(x)
         x=self.func(x)
         x=self.fc2(x)
         x=self.func(x)
         x=self.fc3(x)
         x=self.func(x)
         x=self.fc4(x)
            
         return x
          




In [6]:


start=time.time() 

model=Net() 

device=torch.device("cuda")
model=model.to(device)


optimizer=torch.optim.Adam(model.parameters(),lr=0.001) 

error=torch.nn.CrossEntropyLoss() 

from torch.optim.lr_scheduler import StepLR 
lr=StepLR(optimizer,step_size=2,gamma=0.7)

epoch = 10

kayıp=[] 
count=0
iterasyon=[] 
for i in range(epoch):

    lr.step()

    print("Epoch:",epoch,"LR:",lr.get_lr())

    for i,(images,label) in enumerate(train_loader):
        images=images.to(device)
        label=label.to(device)

        tahmin=model(images)
        optimizer.zero_grad()
        loss=error(tahmin,label)
        loss.backward()
        optimizer.step()

        count+=1

        if count % 100 == 0:
            total=0
            correct=0
            correct_hata=0
            for image,labels in test_loader:
                image=image.to(device)
                labels=labels.to(device)

                out=model(image)
                pred=torch.max(out.data,1)[1]
                total+=len(label)

                correct+=(pred==labels).sum() 
                correct_hata+=(pred!=labels).sum() 
            dogruluk=100*correct/float(total)
            hata=100*correct_hata/float(total)

            kayıp.append(loss.data)
            iterasyon.append(count)

        if count % 100 == 0:
            print('Iteration: {} loss: {} accuracy: {:.2f}% Error: {:.2f}%'.format(count,loss.data,dogruluk,hata))


        

end=time.time()
print("süre:",end-start)

AssertionError: Torch not compiled with CUDA enabled